In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [3]:
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://turbo.az/",
})

In [4]:
url="https://turbo.az/"
response = session.get(url) 

In [5]:
response

<Response [200]>

## MAIN CODE

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_detail_page_links(url):
    
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://turbo.az/",
    })
    
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

   
    item_links = []
    items = soup.find_all('div', class_='products-i') 
    for item in items:
        link_element = item.find('a', href=True) 
        if link_element:
            detail_url = link_element['href']
            item_links.append(f"https://turbo.az{detail_url}") 
    return item_links

def scrape_detail_page(url):
   
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://turbo.az/",
    })
    
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

  
    column1 = soup.find('div', class_='product-properties__column')
    properties1 = column1.find_all('span', class_='product-properties__i-value') if column1 else []

  
    column2 = column1.find_next_sibling('div', class_='product-properties__column') if column1 else None
    additional_properties = column2.find_all('span', class_='product-properties__i-value') if column2 else []

  
    price_element = soup.find('div', class_='product-price__i product-price__i--bold')
    price = price_element.text.strip() if price_element else None

  
    phone_element = soup.find('div', class_='product-phones__btn-value')
    phone = phone_element.text.strip() if phone_element else None
    
    description_element = soup.find('div', class_='product-description__content')
    description = description_element.get_text(strip=True) if description_element else None

   
    city = properties1[0].text.strip() if len(properties1) > 0 else None
    make = properties1[1].find('a').text.strip() if len(properties1) > 1 and properties1[1].find('a') else properties1[1].text.strip() if len(properties1) > 1 else None
    model = properties1[2].find('a').text.strip() if len(properties1) > 2 and properties1[2].find('a') else properties1[2].text.strip() if len(properties1) > 2 else None
    year = properties1[3].find('a').text.strip() if len(properties1) > 3 and properties1[3].find('a') else properties1[3].text.strip() if len(properties1) > 3 else None
    category = properties1[4].text.strip() if len(properties1) > 4 else None
    color = properties1[5].text.strip() if len(properties1) > 5 else None
    engine = properties1[6].text.strip() if len(properties1) > 6 else None
    mileage = properties1[7].text.strip() if len(properties1) > 7 else None

  
    transmission = additional_properties[0].text.strip() if len(additional_properties) > 0 else None
    drivetrain = additional_properties[1].text.strip() if len(additional_properties) > 1 else None
    is_new = additional_properties[2].text.strip() if len(additional_properties) > 2 else None
    seat_count = additional_properties[3].text.strip() if len(additional_properties) > 3 else None
    owners_count = additional_properties[4].text.strip() if len(additional_properties) > 4 else None
    condition = additional_properties[5].text.strip() if len(additional_properties) > 5 else None

   
    return {
        'Şəhər': city,
        'Marka': make,
        'Model': model,
        'Buraxılış ili': year,
        'Ban növü': category,
        'Rəng': color,
        'Mühərrik': engine,
        'Yürüş': mileage,
        'Sürətlər qutusu': transmission,
        'Ötürücü': drivetrain,
        'Yeni': is_new,
        'Yerlərin sayı': seat_count,
        'Sahiblər': owners_count,
        'Vəziyyəti': condition,
        'Qiymət': price,
        'Telefon': phone,  
        'Açıqlama': description, 
    }

base_url = "https://turbo.az/autos?page={}"
all_links = []

for i in range(1, 10):  
    url = base_url.format(i)
    links = get_detail_page_links(url)
    all_links.extend(links)

all_data = []

for link in all_links:
    data = scrape_detail_page(link)
    all_data.append(data)


df = pd.DataFrame(all_data)
print(df)


     Şəhər       Marka      Model Buraxılış ili                 Ban növü  \
0     Bakı         GWM       Poer          2024      Pikap, ikiqat kabin   
1      Qax      C.Moto     CMR-H8          2024               Kvadrosikl   
2     Bakı  Volkswagen     ID.6 X          2022  Offroader / SUV, 5 qapı   
3     Bakı     Changan  Uni-V iDD          2024                  Liftbek   
4     Bakı     Hyundai    Elantra          2017                    Sedan   
..     ...         ...        ...           ...                      ...   
319   Bakı         Kia        Rio          2010          Hetçbek, 5 qapı   
320   Bakı      Toyota      Camry          2018                    Sedan   
321  Qazax  LADA (VAZ)       2107          2007                    Sedan   
322   Bakı         DAF     105 XF          2011                   Dartqı   
323   Bakı  LADA (VAZ)     Granta          2019                    Sedan   

             Rəng                           Mühərrik       Yürüş  \
0         Qəhvəyi  

In [7]:
df

,Şəhər,Marka,Model,Buraxılış ili,Ban növü,Rəng,Mühərrik,Yürüş,Sürətlər qutusu,Ötürücü,Yeni,Yerlərin sayı,Sahiblər,Vəziyyəti,Qiymət,Telefon,Açıqlama
0,Bakı,GWM,Poer,2024,"Pikap, ikiqat kabin",Qəhvəyi,2.0 L / 164 a.g. / Dizel,0 km,Avtomat,Tam,Bəli,5,"Vuruğu yoxdur, rənglənməyib",None,52 500 AZN,*79••,GWM ENDIRIM sevinci var! Seçilmiş avtomobillər...
1,Qax,C.Moto,CMR-H8,2024,Kvadrosikl,Qırmızı,300 sm3 / 48 a.g. / Benzin,0 km,Mexaniki,Arxa,Bəli,2,1,"Vuruğu yoxdur, rənglənməyib",7 300 AZN,(077) 277-23-••,Motor 300cc radiatorlu. yanacaq çəni 23 lt- 4*...
2,Bakı,Volkswagen,ID.6 X,2022,"Offroader / SUV, 5 qapı",Boz,204 a.g. / Elektro,42 400 km,Reduktor,Arxa,Xeyr,7,"Vuruğu yoxdur, rənglənməyib",Çin,25 800 USD,(055) 650-66-••,"Volkswagen ID.6 X, 2022 ilModelin Tam Full kom..."
3,Bakı,Changan,Uni-V iDD,2024,Liftbek,Yaş Asfalt,1.5 L / 325 a.g. / Plug-in Hibrid,0 km,Variator,Ön,Bəli,0,"Vuruğu yoxdur, rənglənməyib",None,37 900 AZN,(077) 470-40-••,Changan Uni-V 2024 Ful Top versionYenilenmiş r...
4,Bakı,Hyundai,Elantra,2017,Sedan,Ağ,2.0 L / 150 a.g. / Benzin,181 856 km,Avtomat,Arxa,Xeyr,5,2,"Vuruğu yoxdur, rənglənməyib",22 500 AZN,(055) 979-93-••,"Avtomabil şəkildə göründüyündən də idealdır, v..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,Bakı,Kia,Rio,2010,"Hetçbek, 5 qapı",Gümüşü,1.5 L / 109 a.g. / Dizel,260 000 km,Mexaniki,Ön,Xeyr,"Vuruğu yoxdur, rənglənməyib",None,None,12 000 AZN,(051) 882-00-••,Salam .mawina raman a da baxa bilersiz iwe eve...
320,Bakı,Toyota,Camry,2018,Sedan,Ağ,2.5 L / 202 a.g. / Benzin,78 858 km,Avtomat,Ön,Xeyr,5,"Vuruğu yoxdur, rənglənməyib",Amerika,33 000 AZN,(077) 303-08-••,Salam. Avtomobil ideal veziyetdedir her bir se...
321,Qazax,LADA (VAZ),2107,2007,Sedan,Tünd qırmızı,1.6 L / 75 a.g. / Benzin,59 840 km,Mexaniki,Arxa,Xeyr,5,3,"Vuruğu yoxdur, rənglənməyib",6 000 AZN,(099) 850-82-••,Yaxşı vəzyəttə
322,Bakı,DAF,105 XF,2011,Dartqı,Ağ,120 L / 460 a.g. / Dizel,975 000 km,Avtomat,Arxa,Xeyr,1,"Vuruğu yoxdur, rənglənməyib",Avropa,81 500 AZN,(055) 646-11-••,"Salam.hər ikisi əla vəziyyətdədir,DAF da maneə..."
